# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#usefull imports
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler

# loading data

In [ ]:
#Path
csv_path = os.path.join(os.getcwd(),  'raw_data')
#files to concatenate
files=['LTE KPIs Part 1.csv','LTE KPIs Part 2.csv','LTE KPIs Part 3.csv','LTE KPIs Part 4.csv']
#Concatanating files
def concat_files(path,liste_files):
# Charger les données du premier fichier CSV
    data=pd.DataFrame()
    for file in liste_files:
        df = pd.read_csv(os.path.join(path, file),sep=";")
        data= pd.concat([data, df], axis=0, ignore_index=True)
    return data


In [ ]:
#add city and city type to dataframe
file='site_city_type.csv'
def add_cities(path,file,dataframe):
    df = pd.read_csv(os.path.join(path,file), sep=";")
    df.rename(columns={"eNodeB ID": "eNodeB identity"}, inplace=True)
    return (dataframe.merge(df, on="eNodeB identity", how="left"))

In [ ]:
data=add_cities(csv_path,file,concat_files(csv_path,files))

In [ ]:
data.head()

In [ ]:
print(data.shape)
print(data.dtypes)

# Transforming numerical values with "," + date processing

In [ ]:
#Transform date from str to date format
data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

In [ ]:
#L.Traffic.User.Avg
#replacing string values by float values
def replace_str_to_float(column_name):
    nb_ligne=data.shape[0]
    new_column=[]
    new_val=0
    for n in range(nb_ligne):
        new_val=data[column_name][n].replace(',','.')
        new_val=new_val.replace('/','')
        new_column.append(float(new_val))
    return new_column

data['Trafic LTE.float']=replace_str_to_float('Trafic LTE')
data['L.Traffic.ActiveUser.Avg.float']=replace_str_to_float('L.Traffic.ActiveUser.Avg')
data['L.Traffic.User.Avg.float']=replace_str_to_float('L.Traffic.User.Avg')
data['DL throughput_GRP.float']=replace_str_to_float('DL throughput_GRP')
data['DL PRB Usage.float']=replace_str_to_float('DL PRB Usage(%)')

In [ ]:
#dropping non numerical values
data.drop(columns=['Trafic LTE','L.Traffic.ActiveUser.Avg','L.Traffic.User.Avg','DL throughput_GRP','DL PRB Usage(%)'], inplace=True)

# scaling the data

In [ ]:
columns_to_scal=['Trafic LTE.float','L.Traffic.ActiveUser.Avg.float','L.Traffic.User.Avg.float','DL throughput_GRP.float','DL PRB Usage.float']
def data_scaling(dataframe,columns_to_scal):
    rb_scaler = RobustScaler()
    for column in columns_to_scal:
        rb_scaler.fit(data[[column]])
        dataframe[column+'.scaled'] = rb_scaler.transform(data[[column]])
    return dataframe

In [ ]:
data=data_scaling(data,columns_to_scal)

In [ ]:
#dropping non scaled values
data.drop(columns=['L.Traffic.ActiveUser.Avg.float','L.Traffic.User.Avg.float','DL throughput_GRP.float','DL PRB Usage.float'], inplace=True)

# Adding missing dates and ID

In [ ]:
data['eNodeB_identifier_int']=(((256*data['eNodeB identity'])+(data['Cell ID'])))
#data['eNodeB_identifier']=data['eNodeB_identifier'].astype(float)
print(data['eNodeB_identifier_int'].value_counts())
type(data['eNodeB_identifier_int'].value_counts())
data['eNodeB_identifier_int'].value_counts().shape


In [ ]:
def replace_missing_dates(df, start_date, end_date) -> pd.DataFrame :
    missing_date=pd.date_range(start = start_date, end = end_date ).difference(df["Date"])
    
    df_new=df.copy()
    if(len(missing_date)>0):
        for i in range(0,len(missing_date)):
            data_sub={'Date': missing_date[i], 
                    'eNodeB identity': df['eNodeB identity'][0], 
                    'Cell ID' : df['Cell ID'][0],
                    'Cell FDD TDD Indication' :df['Cell FDD TDD Indication'][0],
                    'Downlink EARFCN' :df['Downlink EARFCN'][0],
                    'Downlink bandwidth' : df['Downlink bandwidth'][0],
                    'LTECell Tx and Rx Mode': df['LTECell Tx and Rx Mode'][0],
                    'Trafic LTE.float': -10,
                    'L.Traffic.ActiveUser.Avg.float.scaled': -10,
                    'L.Traffic.User.Avg.float.scaled':-10,
                    'DL throughput_GRP.float.scaled': -10,
                    'DL PRB Usage.float.scaled' : -10,
                    'City' : df['City'][0], 
                    'City Type':df['City Type'][0],
                    'eNodeB_identifier_int':df['eNodeB_identifier_int'][0],
                    'Trafic LTE.float.scaled': -10
                    }
            print (data_sub)
            new_row=pd.DataFrame([data_sub])
            df_new=pd.concat([df_new,new_row])
            df_new.sort_values('Date')
    return df_new

In [ ]:
def replace_missing_dates_all(df) -> pd.DataFrame :
    cells=df[['eNodeB_identifier_int']]
    cells=cells.drop_duplicates()
    start_date=df['Date'].min()
    end_date=df['Date'].max()

    for index, row in cells.iterrows():
        df_cell=df[(df["eNodeB_identifier_int"]==row[0])]
        df_cell=df_cell.reset_index(drop=True)

        df_cell=replace_missing_dates(df_cell, start_date, end_date)
        if index==0:
            df_new=df_cell
        else:
            df_new=pd.concat([df_new,df_cell])

    return df_new

In [ ]:
data=replace_missing_dates_all(data)

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.to_csv("data_finale_08_08_2023.csv")

In [ ]:
data=data_new


In [ ]:
data.head()

# Encoding non numerical values

In [ ]:
data['Cell FDD TDD Indication'].value_counts()

In [ ]:
# Check unique values for Cell FDD TDD Indication
print(f"The unique values for 'Downlink EARFCN' are {data['Cell FDD TDD Indication'].unique()}")
# Instantiate the OneHotEncoder
ohe_Cell_FDD_TDD_Indication = OneHotEncoder(sparse = False)
# Fit encoder
ohe_Cell_FDD_TDD_Indication.fit(data[['Cell FDD TDD Indication']])
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe_Cell_FDD_TDD_Indication.categories_}")
# Display the generated names
print(f"The column names for the encoded values are {ohe_Cell_FDD_TDD_Indication.get_feature_names_out()}")
# Transform the current "Street" column
data[ohe_Cell_FDD_TDD_Indication.get_feature_names_out()] = ohe_Cell_FDD_TDD_Indication.transform(data[['Cell FDD TDD Indication']])
# Drop the column "Street" which has been encoded
#data.drop(columns = ['City'], inplace = True)
# Show the dataset
data.head(3)

In [ ]:
data['Downlink EARFCN'].value_counts()

In [ ]:
# Check unique values for Downlink EARFCN Tx and Rx Mode
print(f"The unique values for 'Downlink EARFCN' are {data['Downlink EARFCN'].unique()}")
# Instantiate the OneHotEncoder
ohe_Downlink_EARFCN = OneHotEncoder(sparse = False)
# Fit encoder
ohe_Downlink_EARFCN.fit(data[['Downlink EARFCN']])
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe_Downlink_EARFCN.categories_}")
# Display the generated names
print(f"The column names for the encoded values are {ohe_Downlink_EARFCN.get_feature_names_out()}")
# Transform the current "Street" column
data[ohe_Downlink_EARFCN.get_feature_names_out()] = ohe_Downlink_EARFCN.transform(data[['Downlink EARFCN']])
# Drop the column "Street" which has been encoded
#data.drop(columns = ['City'], inplace = True)
# Show the dataset
data.head(3)

In [ ]:
data['Downlink bandwidth'].value_counts()

In [ ]:
# Check unique values for Downlink bandwidth
print(f"The unique values for 'Downlink bandwidth' are {data['Downlink bandwidth'].unique()}")
# Instantiate the OneHotEncoder
ohe_Downlink_bandwidth = OneHotEncoder(sparse = False)
# Fit encoder
ohe_Downlink_bandwidth.fit(data[['Downlink bandwidth']])
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe_Downlink_bandwidth.categories_}")
# Display the generated names
print(f"The column names for the encoded values are {ohe_Downlink_bandwidth.get_feature_names_out()}")
# Transform the current "Street" column
data[ohe_Downlink_bandwidth.get_feature_names_out()] = ohe_Downlink_bandwidth.transform(data[['Downlink bandwidth']])
# Drop the column "Street" which has been encoded
#data.drop(columns = ['City'], inplace = True)
# Show the dataset
data.head(3)

In [ ]:
data['LTECell Tx and Rx Mode'].value_counts()

In [ ]:
# Check unique values for LTECell Tx and Rx Mode
print(f"The unique values for 'Street' are {data['LTECell Tx and Rx Mode'].unique()}")
# Instantiate the OneHotEncoder
ohe_LTECell_Tx_and_Rx_Mode = OneHotEncoder(sparse = False)
# Fit encoder
ohe_LTECell_Tx_and_Rx_Mode.fit(data[["LTECell Tx and Rx Mode"]])
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe_LTECell_Tx_and_Rx_Mode.categories_}")
# Display the generated names
print(f"The column names for the encoded values are {ohe_LTECell_Tx_and_Rx_Mode.get_feature_names_out()}")
# Transform the current "Street" column
data[ohe_LTECell_Tx_and_Rx_Mode.get_feature_names_out()] = ohe_LTECell_Tx_and_Rx_Mode.transform(data[['LTECell Tx and Rx Mode']])
# Drop the column "Street" which has been encoded
#data.drop(columns = ['City'], inplace = True)
# Show the dataset
data.head(3)

In [ ]:
data['City'].value_counts()

In [ ]:

# Check unique values for city
print(f"The unique values for 'City' are {data.City.unique()}")
# Instantiate the OneHotEncoder
ohe_city = OneHotEncoder(sparse = False)
# Fit encoder
ohe_city.fit(data[["City"]])
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe_city.categories_}")
# Display the generated names
print(f"The column names for the encoded values are {ohe_city.get_feature_names_out()}")
# Transform the current "Street" column
data[ohe_city.get_feature_names_out()] = ohe_city.transform(data[['City']])
# Drop the column "Street" which has been encoded
#data.drop(columns = ['City'], inplace = True)
# Show the dataset
data.head(3)


In [ ]:
#encoding citytype...
data['City Type'].value_counts()

In [ ]:
# Check unique values for City Type
print(f"The unique values for 'City' are {data['City Type'].unique()}")
# Instantiate the OneHotEncoder
ohe_City_Type = OneHotEncoder(sparse = False)
# Fit encoder
ohe_City_Type.fit(data[["City Type"]])
# Display the detected categories
print(f"The categories detected by the OneHotEncoder are {ohe_City_Type.categories_}")
# Display the generated names
print(f"The column names for the encoded values are {ohe_City_Type.get_feature_names_out()}")
# Transform the current "Street" column
data[ohe_City_Type.get_feature_names_out()] = ohe_City_Type.transform(data[['City Type']])
# Drop the column "Street" which has been encoded
#data.drop(columns = ['City'], inplace = True)
# Show the dataset
data.head(3)

In [ ]:
data.head()

In [ ]:
data.drop(columns=['Cell FDD TDD Indication','Downlink EARFCN','Downlink bandwidth','LTECell Tx and Rx Mode','City','City Type'], inplace=True)

In [ ]:
data.to_csv("data_finale_08_08_2023.csv")

In [ ]:
data.dtypes

# Cleaning Data

In [ ]:
data.dtype

In [ ]:
data.head()

# Data scaling

In [ ]:
data.to_csv("data_finale_V4.csv")

In [ ]:
data["L.Traffic.User.Avg"].value_counts().sort_values()
data['eNodeB_identifier_int'].value_counts()
data['eNodeB_identifier_int'].unique

In [ ]:
data_site=data.loc[data['eNodeB identity'] == 146007]
sns.relplot(
    data=data_site, kind="line",
    x='Date', y='L.Traffic.User.Avg.float',col="encoded_Downlink_EARFCN",
    hue="Cell ID",  style="Cell ID"
);

In [ ]:
data.to_csv("data.csv")